### Запуск мінімальної екпертної системи

В консолі CLIPS уже можна добавляти правила, факти та виконувати запуск. При використані *experta* потрібно створити об'єкт який буде симулювати в собі CLIPS 

In [1]:
from experta import KnowledgeEngine

class BasicExample(KnowledgeEngine):
    pass


engine = BasicExample()

В середині об'єкта створюються початкові факти, правила і функції, а запуск еспертної системи відбувається як операція над екземпляром об'єкта 

Запуск ЕС у середовищі CLIPS: 

```CLIPS
(reset)
(run)
```

Запуск ЕС у Python:

In [2]:
engine.reset()
engine.run()

### Створення сутностей, шаблонів та фактів 

В середовищі CLIPS існують *deftemplate* та *defclass* для опису шаблонів фактів та сутностей предметної області. При використані *experta* використовуються лише класи.

Cтворення шаблону animal та person в середовищі CLIPS:
```CLIPS
(deftemplate animal
             (slot name)
             (slot approx_weight)
             (slot approx_height)
             (slot tail)
             (slot character)
             (multislot related_animals))

(deftemplate person
             (slot name)
             (slot animal_kind)
             (slot age)
             (multislot friends)) 

```

Cтворення класу animal та person мовою Python:

In [11]:
from experta import Fact, Field

# Сутність повина наслідуватися від класу experta.Fact
class Animal(Fact):
    # атрибути класу повині бути типу experta.Filed та мати в описаний тип даних
    # тип даних може бути як стандартний для Python, так і бути описаним вручну
    # mandatory вказує на обов'язковість атрибуту в класі 
    name = Field(str, mandatory=True)
    approx_weight = Field(int, mandatory=True)
    approx_height = Field(int, mandatory=True)
    tail = Field(int, mandatory=True)
    character = Field(str, mandatory=True)
    related_animals = Field(list, mandatory=True)
    
    
class Person(Fact):
    name = Field(str, mandatory=False)
    animal_kind = Field(str, mandatory=True)
    age = Field(int, mandatory=True)
    friends = Field(list, mandatory=False)

Створимо декілька початкових фактів для ЕС. Розглянем створення через *deffacts* та *assert* 

CLIPS deffacts:
```CLIPS
(deffacts initfacts "Initial Facts"
             (dogs are our friends)
             (animal (name dog) (approx_weight 20) (approx_height 30) (tail 1)
              (related_animals cat wolf) (character strong)
             )
             (animal (name cat) (approx_weight 4) (approx_height 10) (tail 1)
              (character sweet) (related_animals dog wolf)
             )
             (animal (name wolf) (approx_weight 4) (approx_height 40) (character bad)
             (tail 1)
              (related_animals dog wolf)
             )
)
```

Python deffacts:

In [10]:
from experta import DefFacts

class Example1(KnowledgeEngine):
    @DefFacts()
    def init_pets(self):
        yield Animal(name='dog', approx_weight=20, approx_height=30, tail=1, character='strong', related_animals=['cat', 'wolf'])
        yield Animal(name='cat', approx_weight=4, approx_height=10, tail=1, character='sweet', related_animals=['wolf', 'dog'])      
        yield Animal(name='wolf', approx_weight=4, approx_height=40, tail=1, character='bad', related_animals=['wolf', 'cat'])

CLIPS assert:
```CLIPS
(assert (person (name Rick) (animal_kind dog) (age 3) (friends Tim Kitty)))
(assert (person (name Tim) (animal_kind dog) (age 4) (friends Rick)))
(assert (person (name Kitty) (animal_kind cat) (age 4) (friends Rick)))
(assert (person (animal_kind mollusc) (age 1) (friends)))
```

В Python точно так само зробити не можна, потрібно або створити функцію в середині об'єкта ЕС, або добавити факти в екзепляр ЕС.


Реалізація через функцію:

In [24]:
class Example2(KnowledgeEngine):
    def init_persons(self):
        self.declare(Person(name="Rick", animal_kind='dog', age=3, friends=['Tim', 'Kitty']))
        self.declare(Person(name="Tim", animal_kind='dog', age=4, friends=['Rick']))
        
engine = Example2()
engine.reset()
engine.init_persons()

Додавання фактів через екземпляр ЕС:

In [25]:
engine.declare(Person(name="Kitty", animal_kind='dog', age=4, friends=['Rick'])) 
engine.declare(Person(animal_kind='mollusc', age=1))                   

Person(animal_kind='mollusc', age=1)

Переглянем список фактів
```CLIPS
(facts)
```

Python:

In [26]:
engine.facts

FactList([(0, InitialFact()),
          (1,
           Person(name='Rick', animal_kind='dog', age=3, friends=frozenlist(['Tim', 'Kitty']))),
          (2,
           Person(name='Tim', animal_kind='dog', age=4, friends=frozenlist(['Rick',]))),
          (3,
           Person(name='Kitty', animal_kind='dog', age=4, friends=frozenlist(['Rick',]))),
          (4, Person(animal_kind='mollusc', age=1))])

Видалення фактів за номером в CLIPS:
```CLIPS
(retract 1 3)
```

В Python функція приймає лише одне значення тому прийдеться її викликати двічі для досягнення того ж ефекту:

In [28]:
engine.retract(1)
engine.retract(3) # якщо виконати команду ще раз, то це призведе до помилки бо факти за такими індексам відсутні 

IndexError: Fact not found.